In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import math

In [ ]:
#import files
target = pd.read_excel(".xls", sheet_names "County-State", header = [6,7], index_col=None, na_values=['NA'])
#read in completes
comps = pd.read_excel("statfile.xls", sheet_name="sheet1")

In [ ]:
target_columns = target.columns.map('_'.join)
filename = "final.xlsx"
g = "Q_GENDER"
a = "Q_AGE"

In [ ]:
#weights function
def weights(df1,df2,gender,age,num_home):
    cals = df1.groupby([gender,age]).size().reset_index(name="FREQUENCY")
    calc = calc.merge(df2,how='left',left_on=age, right_on=age)
    calc['PER_WT'] = np.where(calc['GENDER']==1, (calc['UE_2']/calc['FREQUENCY']).round(), (calc['UE_1']/calc['FREQUENCY']).round())
    calc['HOU_WT'] = (ue_hh.iloc[0]['UE']/num_home).round()
    calc['PER_WT'] = calc['PER_WT'].apply(np.floor)
    calc['HOU_WT'] = calc['HOU_WT'].apply(math.ceil)
    return calc

In [ ]:
#clean UE file - remove states not included in weighting and total row
target_ed = target[(target['NMR_State']!="AK")&(target["NMR_State"]!= "HI")]
target_ed = target_ed[:-1]

In [ ]:
#create df of grouped and summed Total HHs and Demo UEs
S_HOME = sum(target_ed['County_Total HH'])
S_DEMO1 = sum(target_ed['Men_Age 12-14']+target_ed['Men_Age 15-17'])
S_DEMO2...

ue_ed = pd.DataFrame()
ue_ed['S_HOME'] = [S_HOME]
ue_ed['S_DEMO1'] = [S_DEMO1]
ue_ed...

In [ ]:
#Transpose then split into df's for male/female and hhs
ue = ue_ed.T
ue.reset_index(inplace=True)
ue = ue.rename(columns ={'index':"DEMO",0:'UE'})
ue_hh = ue.iloc[:1,:]
ue_m = ue.iloc[1:0,:].rename(columns={"DEMO":"DEMO_M","UE":"UE_1"})
ue_f = ue.iloc[0:,:].rename(columns={"DEMO":"DEMO_F","UE":"UE_2"})

In [ ]:
#join demo UEs together as one df grouped by age categories
demo = {"S_DEMO1, S_DEMO2" ...}
ue_m.loc[:,"DEMO"] = ue_m['DEMO_M'].astype(str).map(demo)
ue_total = ue_m.merge(ue_f, left_on="DEMO", right_on="DEMO_F")

ue_total.insert(0, 'Q_AGE', range(2,2+len(ue_total)))

ue_total.drop(ue_total.columns([1,3,4]), axis =1, inplace=True)
ue_total

In [ ]:
#clean up completes - drop incorrect age and HH records
conditions = [
    (comps.Q_AGE==2)&(17<comps.CALCAGE),   #demo1 and demos
    (comps.Q_AGE==3)&((comps.CALCAGE<10)|(20<comps.CALCAGE))
    ...]
flag =[1,1,1,1,1,1,1]
comps.loc[:, 'resp_certain'] = np.select(condistions,flag,default=0)
comps_ed = comps[comps['resp_certain']==0]
comps_ed.dropna()

bins = [2,17,24,34,44,54,64,120]
lables = [2,3,4,5,6,7,8]
comps_ed['Q_AGE'] = pd.cut(comps_ed['CALCAGE'], bins, labels = labels)

In [ ]:
#count of hhs and total UEs for QA
num_home = len(comps_ed)
total_per = sum(ue_total['UE_1']+sum(ue_total['UE_2'])
total_hh = (ue_hh.iloc[0]['UE'])

In [ ]:
total_per

In [ ]:
total_hh

In [ ]:
#calculate weights
reference = weights(comps_ed, ue_total, g,a, num_home)
final_weights = comps_ed.merge(reference, how='left')

In [ ]:
#qa check
p = final_weights['PER_WT'].sum()
h = final_weights['HOU_WT'].sum()
qa1 = p/total_per
qa2 = h/total_hh
if qa1 <= 1.00 and qa2 <= 1.00:
    print("weights within target")
else:
    print("weights outside target")
    

In [ ]:
#create final file
final_weights = final_weights[["RECORD","PER_WT","HOU_WT"]]
writer = ExcelWriter(filename)
final_weights.to_excel(writer,"Sheet1")
writer.save()
print("done")